In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.6.0" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools Plots"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -n "$COLAB_GPU" ] && [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  if [ "$COLAB_GPU" = "1" ]; then
      JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia  

  echo ''
  echo "Success! Please reload this page and jump to the next section."
fi

# Checking the Installation
The `versioninfo()` function should print your Julia version and some other info about the system:

In [ ]:
versioninfo()

Julia Version 1.6.0
Commit f9720dc2eb (2021-03-24 12:55 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: AMD EPYC 7B12
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-11.0.1 (ORCJIT, znver2)
Environment:
  JULIA_NUM_THREADS = 2


In [ ]:
using BenchmarkTools

M = rand(2048, 2048)
@benchmark M^2

BenchmarkTools.Trial: 12 samples with 1 evaluation.
 Range (min … max):  403.438 ms … 496.455 ms  ┊ GC (min … max): 0.00% … 17.67%
 Time  (median):     421.876 ms               ┊ GC (median):    0.06%
 Time  (mean ± σ):   428.216 ms ±  28.366 ms  ┊ GC (mean ± σ):  3.39% ±  6.16%

  ████     █ ████  █                            █             █  
  ████▁▁▁▁▁█▁████▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  403 ms           Histogram: frequency by time          496 ms <

 Memory estimate: 32.00 MiB, allocs estimate: 2.

In [ ]:
if ENV["COLAB_GPU"] == "1"
    using CUDA

    M_gpu = cu(M)
    @benchmark CUDA.@sync M_gpu^2
else
    println("No GPU found.")
end

No GPU found.


In [ ]:
function estimate_pi(n)
    s = 1.0
    for i in 1:n
        s += (isodd(i) ? -1 : 1) / (2i + 1)
    end
    4s
end

p = estimate_pi(100_000_000)
println("π ≈ $p")
println("Error is $(p - π)")

π ≈ 3.141592663589326
Error is 9.999532757376528e-9


In [ ]:
using Pkg

In [ ]:
Packages =["Plots", "StatsBase", "StatsModels","GLPK","DataFrames","JuMP","MathOptInterface","Turing"]

8-element Vector{String}:
 "Plots"
 "StatsBase"
 "StatsModels"
 "GLPK"
 "DataFrames"
 "JuMP"
 "MathOptInterface"
 "Turing"

In [ ]:
Pkg.add(Packages)

   Resolving package versions...
   Installed GLPK_jll ─────── v5.0.1+0
   Installed GLPK ─────────── v0.15.2
   Installed CEnum ────────── v0.4.1
   Installed BinaryProvider ─ v0.5.10
    Updating `~/.julia/environments/v1.6/Project.toml`
  [60bf3e95] + GLPK v0.15.2
    Updating `~/.julia/environments/v1.6/Manifest.toml`
  [b99e7846] + BinaryProvider v0.5.10
  [fa961155] + CEnum v0.4.1
  [60bf3e95] + GLPK v0.15.2
  [e8aa6df9] + GLPK_jll v5.0.1+0
  [781609d7] + GMP_jll
    Building GLPK → `~/.julia/scratchspaces/44cfe95a-1eb2-52ea-b672-e2afdf69b78f/ab6d06aa06ce3de20a82de5f7373b40796260f72/build.log`
Precompiling project...
  ✓ GMP_jll
  ✓ CEnum
  ✓ GLPK_jll
  ✓ BinaryProvider
  ✓ GLPK
5 dependencies successfully precompiled in 9 seconds (267 already precompiled)


In [ ]:
using JuMP

In [ ]:
varinfo(JuMP)

| name                             |        size | summary                                 |
|:-------------------------------- | -----------:|:--------------------------------------- |
| @NLconstraint                    |     0 bytes | JuMP.var"#@NLconstraint"                |
| @NLconstraints                   |     0 bytes | JuMP.var"#@NLconstraints"               |
| @NLexpression                    |     0 bytes | JuMP.var"#@NLexpression"                |
| @NLexpressions                   |     0 bytes | JuMP.var"#@NLexpressions"               |
| @NLobjective                     |     0 bytes | JuMP.var"#@NLobjective"                 |
| @NLparameter                     |     0 bytes | JuMP.var"#@NLparameter"                 |
| @NLparameters                    |     0 bytes | JuMP.var"#@NLparameters"                |
| @SDconstraint                    |     0 bytes | JuMP.var"#@SDconstraint"                |
| @SDconstraints                   |     0 bytes | JuMP.var"#@SDconstraints"               |
| @build_constraint                |     0 bytes | JuMP.var"#@build_constraint"            |
| @constraint                      |     0 bytes | JuMP.var"#@constraint"                  |
| @constraints                     |     0 bytes | JuMP.var"#@constraints"                 |
| @expression                      |     0 bytes | JuMP.var"#@expression"                  |
| @expressions                     |     0 bytes | JuMP.var"#@expressions"                 |
| @objective                       |     0 bytes | JuMP.var"#@objective"                   |
| @variable                        |     0 bytes | JuMP.var"#@variable"                    |
| @variables                       |     0 bytes | JuMP.var"#@variables"                   |
| ALMOST_DUAL_INFEASIBLE           |     4 bytes | MathOptInterface.TerminationStatusCode  |
| ALMOST_INFEASIBLE                |     4 bytes | MathOptInterface.TerminationStatusCode  |
| ALMOST_LOCALLY_SOLVED            |     4 bytes | MathOptInterface.TerminationStatusCode  |
| ALMOST_OPTIMAL                   |     4 bytes | MathOptInterface.TerminationStatusCode  |
| AUTOMATIC                        |     4 bytes | ModelMode                               |
| AbstractConstraint               |   172 bytes | DataType                                |
| AbstractJuMPScalar               |   172 bytes | DataType                                |
| AbstractModel                    |   172 bytes | DataType                                |
| AbstractShape                    |   172 bytes | DataType                                |
| AbstractVariable                 |   172 bytes | DataType                                |
| AbstractVariableRef              |   172 bytes | DataType                                |
| AbstractVectorSet                |   172 bytes | DataType                                |
| AffExpr                          |   212 bytes | DataType                                |
| BinaryRef                        |     0 bytes | typeof(BinaryRef)                       |
| BridgeableConstraint             |    80 bytes | UnionAll                                |
| ConstraintNotOwned               |   160 bytes | UnionAll                                |
| ConstraintRef                    |   120 bytes | UnionAll                                |
| Containers                       | 457.508 KiB | Module                                  |
| DIRECT                           |     4 bytes | ModelMode                               |
| DUAL_INFEASIBLE                  |     4 bytes | MathOptInterface.TerminationStatusCode  |
| FEASIBILITY_SENSE                |     4 bytes | MathOptInterface.OptimizationSense      |
| FEASIBLE_POINT                   |     4 bytes | MathOptInterface.ResultStatusCode       |
| FixRef                           |     0 bytes | typeof(FixRef)                          |
| GenericAffExpr                   |    80 bytes | UnionAll                                |
| GenericQuadExpr                  |    80 bytes | UnionAll                                |
| IJuliaMode                       |   172 bytes | DataType                                |
| INFEASIBILITY_CERTIFICATE        |     4 bytes | MathOptInterface.ResultStatusCode       |
| INFEASIBLE                       |     4 bytes | MathOptInterface.TerminationStatusCode  |
| INFEASIBLE_OR_UNBOUNDED          |     4 bytes | MathOptInterface.TerminationStatusCode  |
| INFEASIBLE_POINT                 |     4 bytes | MathOptInterface.ResultStatusCode       |
| INTERRUPTED                      |     4 bytes | MathOptInterface.TerminationStatusCode  |
| INVALID_MODEL                    |     4 bytes | MathOptInterface.TerminationStatusCode  |
| INVALID_OPTION                   |     4 bytes | MathOptInterface.TerminationStatusCode  |
| ITERATION_LIMIT                  |     4 bytes | MathOptInterface.TerminationStatusCode  |
| IntegerRef                       |     0 bytes | typeof(IntegerRef)                      |
| JuMP                             |   2.433 MiB | Module                                  |
| LOCALLY_INFEASIBLE               |     4 bytes | MathOptInterface.TerminationStatusCode  |
| LOCALLY_SOLVED                   |     4 bytes | MathOptInterface.TerminationStatusCode  |
| LinearTermIterator               |   120 bytes | UnionAll                                |
| LowerBoundRef                    |     0 bytes | typeof(LowerBoundRef)                   |
| MANUAL                           |     4 bytes | ModelMode                               |
| MAX_SENSE                        |     4 bytes | MathOptInterface.OptimizationSense      |
| MEMORY_LIMIT                     |     4 bytes | MathOptInterface.TerminationStatusCode  |
| MIN_SENSE                        |     4 bytes | MathOptInterface.OptimizationSense      |
| MOI                              |  16.347 MiB | Module                                  |
| MOIB                             |   1.122 MiB | Module                                  |
| MOIU                             |   3.096 MiB | Module                                  |
| Model                            |   252 bytes | DataType                                |
| ModelMode                        |   172 bytes | DataType                                |
| NEARLY_FEASIBLE_POINT            |     4 bytes | MathOptInterface.ResultStatusCode       |
| NEARLY_INFEASIBILITY_CERTIFICATE |     4 bytes | MathOptInterface.ResultStatusCode       |
| NEARLY_REDUCTION_CERTIFICATE     |     4 bytes | MathOptInterface.ResultStatusCode       |
| NLPEvaluator                     |   420 bytes | DataType                                |
| NODE_LIMIT                       |     4 bytes | MathOptInterface.TerminationStatusCode  |
| NORM_LIMIT                       |     4 bytes | MathOptInterface.TerminationStatusCode  |
| NO_SOLUTION                      |     4 bytes | MathOptInterface.ResultStatusCode       |
| NUMERICAL_ERROR                  |     4 bytes | MathOptInterface.TerminationStatusCode  |
| NoOptimizer                      |   172 bytes | DataType                                |
| NonlinearConstraintIndex         |   188 bytes | DataType                                |
| NonlinearConstraintRef           |    40 bytes | UnionAll                                |
| NonlinearExpression              |   196 bytes | DataType                                |
| NonlinearParameter               |   196 bytes | DataType                                |
| OBJECTIVE_LIMIT                  |     4 bytes | MathOptInterface.TerminationStatusCode  |
| OPTIMAL                          |     4 bytes | MathOptInterface.TerminationStatusCode  |
| OPTIMIZE_NOT_CALLED              |     4 bytes | MathOptInterface.TerminationStatusCode  |
| OTHER_ERROR                      |     4 bytes | MathOptInterface.TerminationStatusCode  |
| OTHER_LIMIT                      |     4 bytes | MathOptInterface.TerminationStatusCode  |
| OTHER_RESULT_STATUS              |     4 bytes | MathOptInterface.ResultStatusCode       |
| OptimizationSense                |   172 bytes | DataType                                |
| OptimizeNotCalled                |   172 bytes | DataType                                |
| PSDCone                          |   172 bytes | DataType                                |
| PrintMode                        |   172 bytes | DataType                                |
| QuadExpr                         |   212 bytes | DataType                                |
| QuadTermIterator                 |   120 bytes | UnionAll                                |
| REDUCTION_CERTIFICATE            |     4 bytes | MathOptInterface.ResultStatusCode       |
| REPLMode                         |   172 bytes | DataType                                |
| ReferenceMap                     |   196 bytes | DataType                                |
| ResultStatusCode                 |   172 bytes | DataType                                |
| RotatedSecondOrderCone           |   172 bytes | DataType                                |
| SLOW_PROGRESS                    |     4 bytes | MathOptInterface.TerminationStatusCode  |
| SOLUTION_LIMIT                   |     4 bytes | MathOptInterface.TerminationStatusCode  |
| SOS1                             |   188 bytes | DataType                                |
| SOS2                             |   188 bytes | DataType                                |
| ScalarConstraint                 |    80 bytes | UnionAll                                |
| ScalarShape                      |   172 bytes | DataType                                |
| ScalarVariable                   |   160 bytes | UnionAll                                |
| SecondOrderCone                  |   172 bytes | DataType                                |
| SensitivityReport                |   196 bytes | DataType                                |
| SkewSymmetricMatrixShape         |   188 bytes | DataType                                |
| SkewSymmetricMatrixSpace         |   172 bytes | DataType                                |
| SquareMatrixShape                |   188 bytes | DataType                                |
| SymMatrixSpace                   |   172 bytes | DataType                                |
| SymmetricMatrixShape             |   188 bytes | DataType                                |
| TIME_LIMIT                       |     4 bytes | MathOptInterface.TerminationStatusCode  |
| TerminationStatusCode            |   172 bytes | DataType                                |
| UNKNOWN_RESULT_STATUS            |     4 bytes | MathOptInterface.ResultStatusCode       |
| UnorderedPair                    |    40 bytes | UnionAll                                |
| UpperBoundRef                    |     0 bytes | typeof(UpperBoundRef)                   |
| VariableConstrainedOnCreation    |    80 bytes | UnionAll                                |
| VariableInfo                     |   160 bytes | UnionAll                                |
| VariableNotOwned                 |    40 bytes | UnionAll                                |
| VariableRef                      |   196 bytes | DataType                                |
| VariablesConstrainedOnCreation   |   120 bytes | UnionAll                                |
| VectorConstraint                 |   120 bytes | UnionAll                                |
| VectorShape                      |   172 bytes | DataType                                |
| add_NL_constraint                |     0 bytes | typeof(add_NL_constraint)               |
| add_NL_expression                |     0 bytes | typeof(add_NL_expression)               |
| add_bridge                       |     0 bytes | typeof(add_bridge)                      |
| add_constraint                   |     0 bytes | typeof(add_constraint)                  |
| add_to_expression!               |     0 bytes | typeof(add_to_expression!)              |
| add_to_function_constant         |     0 bytes | typeof(add_to_function_constant)        |
| add_variable                     |     0 bytes | typeof(add_variable)                    |
| all_constraints                  |     0 bytes | typeof(all_constraints)                 |
| all_variables                    |     0 bytes | typeof(all_variables)                   |
| backend                          |     0 bytes | typeof(JuMP.backend)                    |
| barrier_iterations               |     0 bytes | typeof(barrier_iterations)              |
| bridge_constraints               |     0 bytes | typeof(bridge_constraints)              |
| build_constraint                 |     0 bytes | typeof(build_constraint)                |
| build_variable                   |     0 bytes | typeof(build_variable)                  |
| callback_node_status             |     0 bytes | typeof(callback_node_status)            |
| callback_value                   |     0 bytes | typeof(callback_value)                  |
| check_belongs_to_model           |     0 bytes | typeof(check_belongs_to_model)          |
| coefficient                      |     0 bytes | typeof(coefficient)                     |
| compute_conflict!                |     0 bytes | typeof(compute_conflict!)               |
| constant                         |     0 bytes | typeof(constant)                        |
| constraint_by_name               |     0 bytes | typeof(constraint_by_name)              |
| constraint_object                |     0 bytes | typeof(constraint_object)               |
| constraint_ref_with_index        |     0 bytes | typeof(constraint_ref_with_index)       |
| constraint_string                |     0 bytes | typeof(constraint_string)               |
| constraints_string               |     0 bytes | typeof(constraints_string)              |
| copy_conflict                    |     0 bytes | typeof(copy_conflict)                   |
| copy_extension_data              |     0 bytes | typeof(copy_extension_data)             |
| copy_model                       |     0 bytes | typeof(copy_model)                      |
| delete                           |     0 bytes | typeof(delete)                          |
| delete_lower_bound               |     0 bytes | typeof(delete_lower_bound)              |
| delete_upper_bound               |     0 bytes | typeof(delete_upper_bound)              |
| direct_model                     |     0 bytes | typeof(direct_model)                    |
| drop_zeros!                      |     0 bytes | typeof(drop_zeros!)                     |
| dual                             |     0 bytes | typeof(dual)                            |
| dual_objective_value             |     0 bytes | typeof(dual_objective_value)            |
| dual_shape                       |     0 bytes | typeof(dual_shape)                      |
| dual_start_value                 |     0 bytes | typeof(dual_start_value)                |
| dual_status                      |     0 bytes | typeof(dual_status)                     |
| error_if_direct_mode             |     0 bytes | typeof(error_if_direct_mode)            |
| fix                              |     0 bytes | typeof(fix)                             |
| fix_value                        |     0 bytes | typeof(fix_value)                       |
| function_string                  |     0 bytes | typeof(function_string)                 |
| get_optimizer_attribute          |     0 bytes | typeof(get_optimizer_attribute)         |
| getdual                          |     0 bytes | typeof(getdual)                         |
| getobjectivebound                |     0 bytes | typeof(getobjectivebound)               |
| getobjectivevalue                |     0 bytes | typeof(getobjectivevalue)               |
| getvalue                         |     0 bytes | typeof(getvalue)                        |
| has_duals                        |     0 bytes | typeof(has_duals)                       |
| has_lower_bound                  |     0 bytes | typeof(has_lower_bound)                 |
| has_upper_bound                  |     0 bytes | typeof(has_upper_bound)                 |
| has_values                       |     0 bytes | typeof(has_values)                      |
| in_set_string                    |     0 bytes | typeof(in_set_string)                   |
| index                            |     0 bytes | typeof(index)                           |
| is_binary                        |     0 bytes | typeof(is_binary)                       |
| is_fixed                         |     0 bytes | typeof(is_fixed)                        |
| is_integer                       |     0 bytes | typeof(is_integer)                      |
| is_valid                         |     0 bytes | typeof(is_valid)                        |
| isequal_canonical                |     0 bytes | typeof(isequal_canonical)               |
| jump_function                    |     0 bytes | typeof(jump_function)                   |
| jump_function_type               |     0 bytes | typeof(jump_function_type)              |
| latex_formulation                |     0 bytes | typeof(latex_formulation)               |
| linear_terms                     |     0 bytes | typeof(linear_terms)                    |
| linearterms                      |     0 bytes | typeof(linearterms)                     |
| list_of_constraint_types         |     0 bytes | typeof(list_of_constraint_types)        |
| lower_bound                      |     0 bytes | typeof(lower_bound)                     |
| lp_sensitivity_report            |     0 bytes | typeof(lp_sensitivity_report)           |
| map_coefficients                 |     0 bytes | typeof(map_coefficients)                |
| map_coefficients_inplace!        |     0 bytes | typeof(map_coefficients_inplace!)       |
| mode                             |     0 bytes | typeof(JuMP.mode)                       |
| model_string                     |     0 bytes | typeof(model_string)                    |
| moi_function                     |     0 bytes | typeof(moi_function)                    |
| moi_function_type                |     0 bytes | typeof(moi_function_type)               |
| moi_set                          |     0 bytes | typeof(moi_set)                         |
| name                             |     0 bytes | typeof(name)                            |
| nl_constraint_string             |     0 bytes | typeof(nl_constraint_string)            |
| nl_expr_string                   |     0 bytes | typeof(nl_expr_string)                  |
| node_count                       |     0 bytes | typeof(node_count)                      |
| normalized_coefficient           |     0 bytes | typeof(normalized_coefficient)          |
| normalized_rhs                   |     0 bytes | typeof(normalized_rhs)                  |
| num_constraints                  |     0 bytes | typeof(num_constraints)                 |
| num_nl_constraints               |     0 bytes | typeof(num_nl_constraints)              |
| num_variables                    |     0 bytes | typeof(num_variables)                   |
| numnlconstr                      |     0 bytes | typeof(numnlconstr)                     |
| numvar                           |     0 bytes | typeof(numvar)                          |
| object_dictionary                |     0 bytes | typeof(object_dictionary)               |
| objective_bound                  |     0 bytes | typeof(objective_bound)                 |
| objective_function               |     0 bytes | typeof(objective_function)              |
| objective_function_string        |     0 bytes | typeof(objective_function_string)       |
| objective_function_type          |     0 bytes | typeof(objective_function_type)         |
| objective_sense                  |     0 bytes | typeof(objective_sense)                 |
| objective_value                  |     0 bytes | typeof(objective_value)                 |
| op_hint                          |    98 bytes | 90-codeunit String                      |
| operator_to_set                  |     0 bytes | typeof(operator_to_set)                 |
| operator_warn                    |     0 bytes | typeof(operator_warn)                   |
| optimize!                        |     0 bytes | typeof(optimize!)                       |
| optimizer_index                  |     0 bytes | typeof(optimizer_index)                 |
| optimizer_with_attributes        |     0 bytes | typeof(optimizer_with_attributes)       |
| owner_model                      |     0 bytes | typeof(owner_model)                     |
| parse_SD_constraint              |     0 bytes | typeof(parse_SD_constraint)             |
| parse_SD_constraint_expr         |     0 bytes | typeof(parse_SD_constraint_expr)        |
| parse_constraint                 |     0 bytes | typeof(parse_constraint)                |
| parse_constraint_call            |     0 bytes | typeof(parse_constraint_call)           |
| parse_constraint_expr            |     0 bytes | typeof(parse_constraint_expr)           |
| parse_constraint_head            |     0 bytes | typeof(parse_constraint_head)           |
| parse_one_operator_constraint    |     0 bytes | typeof(parse_one_operator_constraint)   |
| parse_one_operator_variable      |     0 bytes | typeof(parse_one_operator_variable)     |
| parse_ternary_variable           |     0 bytes | typeof(parse_ternary_variable)          |
| parse_variable                   |     0 bytes | typeof(parse_variable)                  |
| primal_feasibility_report        |     0 bytes | typeof(primal_feasibility_report)       |
| primal_status                    |     0 bytes | typeof(primal_status)                   |
| print_bridge_graph               |     0 bytes | typeof(print_bridge_graph)              |
| quad_terms                       |     0 bytes | typeof(quad_terms)                      |
| raw_status                       |     0 bytes | typeof(raw_status)                      |
| read_from_file                   |     0 bytes | typeof(read_from_file)                  |
| reduced_cost                     |     0 bytes | typeof(reduced_cost)                    |
| register                         |     0 bytes | typeof(register)                        |
| relative_gap                     |     0 bytes | typeof(relative_gap)                    |
| relax_integrality                |     0 bytes | typeof(relax_integrality)               |
| reshape_set                      |     0 bytes | typeof(reshape_set)                     |
| reshape_vector                   |     0 bytes | typeof(reshape_vector)                  |
| result_count                     |     0 bytes | typeof(result_count)                    |
| reverse_sense                    |     0 bytes | typeof(reverse_sense)                   |
| sense_to_set                     |     0 bytes | typeof(sense_to_set)                    |
| set_NL_objective                 |     0 bytes | typeof(set_NL_objective)                |
| set_binary                       |     0 bytes | typeof(set_binary)                      |
| set_coefficient                  |     0 bytes | typeof(set_coefficient)                 |
| set_dual_start_value             |     0 bytes | typeof(set_dual_start_value)            |
| set_integer                      |     0 bytes | typeof(set_integer)                     |
| set_lower_bound                  |     0 bytes | typeof(set_lower_bound)                 |
| set_name                         |     0 bytes | typeof(set_name)                        |
| set_normalized_coefficient       |     0 bytes | typeof(set_normalized_coefficient)      |
| set_normalized_rhs               |     0 bytes | typeof(set_normalized_rhs)              |
| set_objective                    |     0 bytes | typeof(set_objective)                   |
| set_objective_coefficient        |     0 bytes | typeof(set_objective_coefficient)       |
| set_objective_function           |     0 bytes | typeof(set_objective_function)          |
| set_objective_sense              |     0 bytes | typeof(set_objective_sense)             |
| set_optimize_hook                |     0 bytes | typeof(set_optimize_hook)               |
| set_optimizer                    |     0 bytes | typeof(set_optimizer)                   |
| set_optimizer_attribute          |     0 bytes | typeof(set_optimizer_attribute)         |
| set_optimizer_attributes         |     0 bytes | typeof(set_optimizer_attributes)        |
| set_parameter                    |     0 bytes | typeof(set_parameter)                   |
| set_parameters                   |     0 bytes | typeof(set_parameters)                  |
| set_silent                       |     0 bytes | typeof(set_silent)                      |
| set_start_value                  |     0 bytes | typeof(set_start_value)                 |
| set_time_limit_sec               |     0 bytes | typeof(set_time_limit_sec)              |
| set_upper_bound                  |     0 bytes | typeof(set_upper_bound)                 |
| set_value                        |     0 bytes | typeof(set_value)                       |
| setlowerbound                    |     0 bytes | typeof(setlowerbound)                   |
| setupperbound                    |     0 bytes | typeof(setupperbound)                   |
| setvalue                         |     0 bytes | typeof(setvalue)                        |
| shadow_price                     |     0 bytes | typeof(shadow_price)                    |
| shape                            |     0 bytes | typeof(JuMP.shape)                      |
| show_backend_summary             |     0 bytes | typeof(show_backend_summary)            |
| show_constraints_summary         |     0 bytes | typeof(show_constraints_summary)        |
| show_objective_function_summary  |     0 bytes | typeof(show_objective_function_summary) |
| simplex_iterations               |     0 bytes | typeof(simplex_iterations)              |
| solution_summary                 |     0 bytes | typeof(solution_summary)                |
| solve                            |     0 bytes | typeof(solve)                           |
| solve_time                       |     0 bytes | typeof(solve_time)                      |
| solver_name                      |     0 bytes | typeof(solver_name)                     |
| start_value                      |     0 bytes | typeof(start_value)                     |
| termination_status               |     0 bytes | typeof(termination_status)              |
| time_limit_sec                   |     0 bytes | typeof(time_limit_sec)                  |
| unfix                            |     0 bytes | typeof(unfix)                           |
| unregister                       |     0 bytes | typeof(unregister)                      |
| unsafe_backend                   |     0 bytes | typeof(unsafe_backend)                  |
| unset_binary                     |     0 bytes | typeof(unset_binary)                    |
| unset_integer                    |     0 bytes | typeof(unset_integer)                   |
| unset_silent                     |     0 bytes | typeof(unset_silent)                    |
| unset_time_limit_sec             |     0 bytes | typeof(unset_time_limit_sec)            |
| upper_bound                      |     0 bytes | typeof(upper_bound)                     |
| value                            |     0 bytes | typeof(JuMP.value)                      |
| variable_by_name                 |     0 bytes | typeof(variable_by_name)                |
| variable_ref_type                |     0 bytes | typeof(variable_ref_type)               |
| vectorize                        |     0 bytes | typeof(vectorize)                       |
| with_optimizer                   |     0 bytes | typeof(with_optimizer)                  |
| writeLP                          |     0 bytes | typeof(writeLP)                         |
| writeMPS                         |     0 bytes | typeof(writeMPS)                        |
| write_to_file                    |     0 bytes | typeof(write_to_file)                   |


In [ ]:
using GLPK

In [ ]:
model=Model(GLPK.Optimizer)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK

In [ ]:
@variable(model, x>=0)
@variable(model,y>=0)

y

In [ ]:
@constraint(model, x + y <= 10)
@constraint(model, x-y>=3)
@constraint(model,5*x +4*y <=35)

5 x + 4 y ≤ 35.0

In [ ]:
@objective(model,Max, 5x+ 6y)

5 x + 6 y

In [ ]:
@show model

model = A JuMP Model
Maximization problem with:
Variables: 2
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 1 constraint
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 2 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 2 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK
Names registered in the model: x, y


A JuMP Model
Maximization problem with:
Variables: 2
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 1 constraint
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 2 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 2 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK
Names registered in the model: x, y

In [ ]:
optimize!(model)

In [ ]:
termination_status(model)

OPTIMAL::TerminationStatusCode = 1

In [ ]:
display(typeof(MOI.OPTIMAL))

Enum MathOptInterface.TerminationStatusCode:
OPTIMIZE_NOT_CALLED = 0
OPTIMAL = 1
INFEASIBLE = 2
DUAL_INFEASIBLE = 3
LOCALLY_SOLVED = 4
LOCALLY_INFEASIBLE = 5
INFEASIBLE_OR_UNBOUNDED = 6
ALMOST_OPTIMAL = 7
ALMOST_INFEASIBLE = 8
ALMOST_DUAL_INFEASIBLE = 9
ALMOST_LOCALLY_SOLVED = 10
ITERATION_LIMIT = 11
TIME_LIMIT = 12
NODE_LIMIT = 13
SOLUTION_LIMIT = 14
MEMORY_LIMIT = 15
OBJECTIVE_LIMIT = 16
NORM_LIMIT = 17
OTHER_LIMIT = 18
SLOW_PROGRESS = 19
NUMERICAL_ERROR = 20
INVALID_MODEL = 21
INVALID_OPTION = 22
INTERRUPTED = 23
OTHER_ERROR = 24

In [ ]:
@show objective_value(model)

objective_value(model) = 39.44444444444444


39.44444444444444